# Pandas Dataframes

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 8)
plt.rcParams['figure.figsize'] = (9, 6)

## Create a [DataFrame](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe)

In [2]:
dates = pd.date_range('20130101', periods=6)
pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))

,A,B,C,D
2013-01-01,1.304053,0.039070,0.812933,1.504558
2013-01-02,0.237534,-1.471841,0.411851,0.379634
2013-01-03,-0.056575,-0.460032,-0.990638,-1.224393
2013-01-04,1.531423,-0.076125,-0.147251,0.907468
2013-01-05,0.339712,0.315319,0.871131,1.013255
2013-01-06,0.199543,0.612715,-0.923262,1.875674


In [3]:
pd.DataFrame({'A' : 1.,
              'B' : pd.Timestamp('20130102'),
              'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
              'D' : np.arange(4,dtype='int32'),
              'E' : pd.Categorical(["test","train","test","train"]),
              'F' : 'foo' })

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,0,test,foo
1,1.0,2013-01-02,1.0,1,train,foo
2,1.0,2013-01-02,1.0,2,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


## Load Data from CSV File

In [4]:
url = "https://www.fun-mooc.fr/c4x/agrocampusouest/40001S03/asset/AnaDo_JeuDonnees_TemperatFrance.csv"
french_cities = pd.read_csv(url, delimiter=";", encoding="latin1", index_col=0)
french_cities

,Janv,Févr,Mars,Avri,Mai,Juin,juil,Août,Sept,Octo,Nove,Déce,Lati,Long,Moye,Ampl,Région
Bordeaux,5.6,6.6,10.3,12.8,15.8,19.3,20.9,21.0,18.6,13.8,9.1,6.2,44.50,-0.34,13.33,15.4,SO
Brest,6.1,5.8,7.8,9.2,11.6,14.4,15.6,16.0,14.7,12.0,9.0,7.0,48.24,-4.29,10.77,10.2,NO
Clermont,2.6,3.7,7.5,10.3,13.8,17.3,19.4,19.1,16.2,11.2,6.6,3.6,45.47,3.05,10.94,16.8,SE
Grenoble,1.5,3.2,7.7,10.6,14.5,17.8,20.1,19.5,16.7,11.4,6.5,2.3,45.10,5.43,10.98,18.6,SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Rennes,4.8,5.3,7.9,10.1,13.1,16.2,17.9,17.8,15.7,11.6,7.8,5.4,48.05,-1.41,11.13,13.1,NO
Strasbourg,0.4,1.5,5.6,9.8,14.0,17.2,19.0,18.3,15.1,9.5,4.9,1.3,48.35,7.45,9.72,18.6,NE
Toulouse,4.7,5.6,9.2,11.6,14.9,18.7,20.9,20.9,18.3,13.3,8.6,5.5,43.36,1.26,12.68,16.2,SO
Vichy,2.4,3.4,7.1,9.9,13.6,17.1,19.3,18.8,16.0,11.0,6.6,3.4,46.08,3.26,10.72,16.9,SE


## Viewing Data

In [5]:
french_cities.head()

,Janv,Févr,Mars,Avri,Mai,Juin,juil,Août,Sept,Octo,Nove,Déce,Lati,Long,Moye,Ampl,Région
Bordeaux,5.6,6.6,10.3,12.8,15.8,19.3,20.9,21.0,18.6,13.8,9.1,6.2,44.50,-0.34,13.33,15.4,SO
Brest,6.1,5.8,7.8,9.2,11.6,14.4,15.6,16.0,14.7,12.0,9.0,7.0,48.24,-4.29,10.77,10.2,NO
Clermont,2.6,3.7,7.5,10.3,13.8,17.3,19.4,19.1,16.2,11.2,6.6,3.6,45.47,3.05,10.94,16.8,SE
Grenoble,1.5,3.2,7.7,10.6,14.5,17.8,20.1,19.5,16.7,11.4,6.5,2.3,45.10,5.43,10.98,18.6,SE
Lille,2.4,2.9,6.0,8.9,12.4,15.3,17.1,17.1,14.7,10.4,6.1,3.5,50.38,3.04,9.73,14.7,NE


In [6]:
french_cities.tail()

,Janv,Févr,Mars,Avri,Mai,Juin,juil,Août,Sept,Octo,Nove,Déce,Lati,Long,Moye,Ampl,Région
Paris,3.4,4.1,7.6,10.7,14.3,17.5,19.1,18.7,16.0,11.4,7.1,4.3,48.52,2.20,11.18,15.7,NE
Rennes,4.8,5.3,7.9,10.1,13.1,16.2,17.9,17.8,15.7,11.6,7.8,5.4,48.05,-1.41,11.13,13.1,NO
Strasbourg,0.4,1.5,5.6,9.8,14.0,17.2,19.0,18.3,15.1,9.5,4.9,1.3,48.35,7.45,9.72,18.6,NE
Toulouse,4.7,5.6,9.2,11.6,14.9,18.7,20.9,20.9,18.3,13.3,8.6,5.5,43.36,1.26,12.68,16.2,SO
Vichy,2.4,3.4,7.1,9.9,13.6,17.1,19.3,18.8,16.0,11.0,6.6,3.4,46.08,3.26,10.72,16.9,SE


## Index

In [7]:
french_cities.index

Index(['Bordeaux', 'Brest', 'Clermont', 'Grenoble', 'Lille', 'Lyon',
       'Marseille', 'Montpellier', 'Nantes', 'Nice', 'Paris', 'Rennes',
       'Strasbourg', 'Toulouse', 'Vichy'],
      dtype='object')

We can rename an index by setting its name.

In [8]:
french_cities.index.name = "City"
french_cities.head()

,Janv,Févr,Mars,Avri,Mai,Juin,juil,Août,Sept,Octo,Nove,Déce,Lati,Long,Moye,Ampl,Région
City,,,,,,,,,,,,,,,,,
Bordeaux,5.6,6.6,10.3,12.8,15.8,19.3,20.9,21.0,18.6,13.8,9.1,6.2,44.50,-0.34,13.33,15.4,SO
Brest,6.1,5.8,7.8,9.2,11.6,14.4,15.6,16.0,14.7,12.0,9.0,7.0,48.24,-4.29,10.77,10.2,NO
Clermont,2.6,3.7,7.5,10.3,13.8,17.3,19.4,19.1,16.2,11.2,6.6,3.6,45.47,3.05,10.94,16.8,SE
Grenoble,1.5,3.2,7.7,10.6,14.5,17.8,20.1,19.5,16.7,11.4,6.5,2.3,45.10,5.43,10.98,18.6,SE
Lille,2.4,2.9,6.0,8.9,12.4,15.3,17.1,17.1,14.7,10.4,6.1,3.5,50.38,3.04,9.73,14.7,NE


### Exercise: Rename DataFrame Months in English

## From a local or remote HTML file
We can download and extract data about mean sea level stations around the world from the [PSMSL website](http://www.psmsl.org/).

In [9]:
# Needs `lxml`, `beautifulSoup4` and `html5lib` python packages
table_list = pd.read_html("http://www.psmsl.org/data/obtaining/")

In [10]:
# there is 1 table on that page which contains metadata about the stations where 
# sea levels are recorded
local_sea_level_stations = table_list[0]
local_sea_level_stations

,Station Name,ID,Lat.,Lon.,GLOSS ID,Country,Date,Coastline,Station
0,BREST,1,48.383,-4.495,242.0,FRA,07/08/2019,190,91
1,SWINOUJSCIE,2,53.917,14.233,NaN,POL,19/10/2001,110,92
2,SHEERNESS,3,51.446,0.743,NaN,GBR,06/06/2019,170,101
3,HOLYHEAD,5,53.314,-4.620,NaN,GBR,13/08/2020,170,191
...,...,...,...,...,...,...,...,...,...
1544,SUVA-B,2356,-18.133,178.428,NaN,FJI,28/01/2020,742,14
1545,SYDNEY PORT JACKSON,2358,-33.826,151.259,NaN,AUS,13/06/2019,680,138
1546,ARKO,2359,58.484,16.961,NaN,SWE,25/08/2020,50,112
1547,UDDEVALLA,2360,58.348,11.895,NaN,SWE,25/08/2020,50,22


## Indexing on DataFrames

In [11]:
french_cities['Lati']  # DF [] accesses columns (Series)

City
Bordeaux      44.50
Brest         48.24
Clermont      45.47
Grenoble      45.10
              ...  
Rennes        48.05
Strasbourg    48.35
Toulouse      43.36
Vichy         46.08
Name: Lati, Length: 15, dtype: float64

`.loc` and `.iloc` allow to access individual values, slices or masked selections:

In [12]:
french_cities.loc['Rennes', "Sep"]

KeyError: 'Sep'

In [ ]:
french_cities.loc['Rennes', ["Sep", "Dec"]]

In [ ]:
french_cities.loc['Rennes', "Sep":"Dec"]

## Masking

In [ ]:
mask = [True, False] * 6 + 5 * [False]
print(french_cities.iloc[:, mask])

In [ ]:
print(french_cities.loc["Rennes", mask])

## New column

In [ ]:
french_cities["std"] = french_cities.iloc[:,:12].std(axis=1)
french_cities

In [ ]:
french_cities = french_cities.drop("std", axis=1) # remove this new column

In [ ]:
french_cities

## Modifying a dataframe with multiple indexing

In [ ]:
# french_cities['Rennes']['Sep'] = 25 # It does not works and breaks the DataFrame
french_cities.loc['Rennes']['Sep'] # = 25 is the right way to do it

In [ ]:
french_cities

## Transforming datasets

In [ ]:
french_cities['Mean'].min(), french_cities['Ampl'].max()

## Apply

Let's convert the temperature mean from Celsius to Fahrenheit degree.

In [ ]:
fahrenheit = lambda T: T*9/5+32
french_cities['Mean'].apply(fahrenheit)

## Sort

In [ ]:
french_cities.sort_values(by='Lati')

In [ ]:
french_cities = french_cities.sort_values(by='Lati',ascending=False)
french_cities

## Stack and unstack

Instead of seeing the months along the axis 1, and the cities along the axis 0, let's try to convert these into an outer and an inner axis along only 1 time dimension.

In [ ]:
pd.set_option("display.max_rows", 20)
unstacked = french_cities.iloc[:,:12].unstack()
unstacked

In [ ]:
type(unstacked)

## Transpose

The result is grouped in the wrong order since it sorts first the axis that was unstacked. We need to transpose the dataframe.

In [ ]:
city_temp = french_cities.iloc[:,:12].transpose()
city_temp.plot()

In [ ]:
city_temp.boxplot(rot=90);

## Describing

In [ ]:
french_cities['Région'].describe()

In [ ]:
french_cities['Région'].unique()

In [ ]:
french_cities['Région'].value_counts()

In [ ]:
# To save memory, we can convert it to a categorical column:
french_cities["Région"] = french_cities["Région"].astype("category")

In [ ]:
french_cities.memory_usage()

## Data Aggregation/summarization

## groupby

In [ ]:
fc_grouped_region = french_cities.groupby("Région")
type(fc_grouped_region)

In [ ]:
for group_name, subdf in fc_grouped_region:
    print(group_name)
    print(subdf)
    print("")

### Exercise

Consider the following dataset [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant).
This dataset consists of records of measurements relating to peaker power plants of 10000 points over 6 years (2006-2011).

**Variables**
- AT = Atmospheric Temperature in C
- V = Exhaust Vaccum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output

We want to model the power output as a function of the other parameters.

Observations are in 5 excel sheets of about 10000 records in "Folds5x2_pp.xlsx". These 5 sheets are same data shuffled.
- Read this file with the pandas function [read_excel](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html). What is the type returned by this function?
- Implement a `select` function to regroup all observations in a pandas serie.
- Use `select` function and `corr` to compute the maximum correlation.
- Parallelize this loop with `concurrent.futures`.